In [1]:
Text cell <9rNQk0qAjAuQ>
# %% [markdown]
# **Clasificación de imágenes (CIFAR-10 data-set)**
En este Jupyter Notebook mostraremos cómo clasificar un dat-set de 60,000 imágenes para su correcta clasificación. Nos encontramos ante un claro ejemplo de clasificación, para lo cual haremos uso de una red profunda convolucional que terminará en una capa de salida densa fully-connected del tamaño del número de etiquetas del que consta nuestro data-set.

Text cell <BGgyxqNJjGz1>
# %% [markdown]
##Importar Líbrerias

Code cell <M8oehRd9Up0y>
# %% [code]
import os
#import time

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from keras.models import load_model

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt

import cv2
from google.colab.patches import cv2_imshow

import warnings
warnings.filterwarnings('ignore')


Text cell <wN0JptxhjLIT>
# %% [markdown]
##Cargar conjunto de datos
1. El data-set que vamos a usar ocupa aproximadamente 170 MB y se descargará automáticamente desde los tutoriales de ejemplo de Keras.
2. En el CIFAR-10 data-set contamos con imágenes de 10 clases distintas: aviones, coches, pájaros, perros, gatos, cérvidos, ranas, equinos, barcos y camiones.

- [tensorflow datasets](https://www.tensorflow.org/datasets/catalog/cifar10?hl=es-419)

Code cell <67jQ01WsU0r8>
# %% [code]
from keras.datasets import cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

Code cell <oRVgoNRCU6rl>
# %% [code]
num_train, img_rows, img_cols,  img_channels =  train_images.shape
num_train , img_rows, img_cols,  img_channels
Execution output
0KB
	text/plain
		(50000, 32, 32, 3)

Code cell <_njEPhHvVvGs>
# %% [code]
num_test, _, _, _ =  test_images.shape
num_test, img_channels, img_rows, img_cols
Execution output
0KB
	text/plain
		(10000, 3, 32, 32)

Code cell <OL6TR_LBV2HD>
# %% [code]
class_names = ['airplane','automobile','bird','cat','deer', 'dog','frog','horse','ship','truck']

Text cell <S_mNUjkgjPfj>
# %% [markdown]
##Mostrar imagenes del conjunto de datos



Code cell <DdmXd6PQV40k>
# %% [code]
fig = plt.figure(figsize=(8,3))
for i in range(len(class_names)):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    idx = np.where(train_labels == i)[0]
    features_idx = train_images[idx]
    img_num = np.random.randint(features_idx.shape[0])
    im = features_idx[img_num]
    ax.set_title(class_names[i])
    plt.imshow(im)
plt.show()
Execution output
68KB
	text/plain
		<Figure size 800x300 with 10 Axes>

Code cell <JIN5loL9V-AO>
# %% [code]
# Normalizamos el train y el test data-set entre 0 y 1
train_features = train_images.astype('float32') / 255.
test_features = test_images.astype('float32') / 255.

# Convertimos las etiquetas a variables One-Hot Encoded

num_classes = len(np.unique(train_labels))

train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

Code cell <NLM5JwJfiaZs>
# %% [code]
train_labels
Execution output
0KB
	text/plain
		array([[0., 0., 0., ..., 0., 0., 0.],
		       [0., 0., 0., ..., 0., 0., 1.],
		       [0., 0., 0., ..., 0., 0., 1.],
		       ...,
		       [0., 0., 0., ..., 0., 0., 1.],
		       [0., 1., 0., ..., 0., 0., 0.],
		       [0., 1., 0., ..., 0., 0., 0.]])

Text cell <BZ9NwuUxjW68>
# %% [markdown]
##Creación del modelo convolucional

Code cell <9U0-bFTGXGXp>
# %% [code]
train_features.shape[1:]
Execution output
0KB
	text/plain
		(32, 32, 3)

Code cell <RALEr6PoWPrS>
# %% [code]
# Definición del modelo
# Iniciamos el modelo de manera secuencial
model = Sequential()
# Continuamos añadiendo al modelo las capas sin preocuparnos de la dimensionalidad de los inputs
# salvo en la primera capa
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu',
                 input_shape=train_features.shape[1:]))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(rate=0.25))
# Hacemos un flattening de la última capa de Pooling
model.add(Flatten())

model.add(Dense(units=512, activation = 'relu'))

model.add(Dropout(rate=0.25))

model.add(Dense(units=num_classes, activation='softmax'))

Code cell <j0xj5EQEW-3a>
# %% [code]
# Compilación del modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Code cell <AP8s0snbW_9y>
# %% [code]
model.fit(train_features, train_labels, epochs=5)
Execution output
1KB
	Stream
		Epoch 1/5
		[1m1563/1563[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m247s[0m 158ms/step - accuracy: 0.6290 - loss: 1.0431
		Epoch 2/5
		[1m1563/1563[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m254s[0m 153ms/step - accuracy: 0.6917 - loss: 0.8703
		Epoch 3/5
		[1m1563/1563[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m263s[0m 153ms/step - accuracy: 0.7285 - loss: 0.7662
		Epoch 4/5
		[1m1563/1563[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m261s[0m 153ms/step - accuracy: 0.7574 - loss: 0.6915
		Epoch 5/5
		[1m1563/1563[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m268s[0m 157ms/step - accuracy: 0.7787 - loss: 0.6252
	text/plain
		<keras.src.callbacks.history.History at 0x7c7bf5a73cd0>

Text cell <cZoiU91Yjf8L>
# %% [markdown]
##Guardar el modelo entrenado

Code cell <aQq5X0D8Y4jk>
# %% [code]
model.save('cifar10_model.keras')

Code cell <ZaGSXlKEY80v>
# %% [code]
from keras.models import load_model
loaded_model = load_model('cifar10_model.keras')

Code cell <jRJNlMY8eGbX>
# %% [code]
loaded_model.summary()
Execution output
13KB
	text/plain
		[1mModel: "sequential_1"[0m
		┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
		┃[1m [0m[1mLayer (type)                        [0m[1m [0m┃[1m [0m[1mOutput Shape               [0m[1m [0m┃[1m [0m[1m        Param #[0m[1m [0m┃
		┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
		│ conv2d_4 ([38;5;33mConv2D[0m)                    │ ([38;5;45mNone[0m, [38;5;34m32[0m, [38;5;34m32[0m, [38;5;34m32[0m)          │             [38;5;34m896[0m │
		├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
		│ conv2d_5 ([38;5;33mConv2D[0m)                    │ ([38;5;45mNone[0m, [38;5;34m30[0m, [38;5;34m30[0m, [38;5;34m32[0m)          │           [38;5;34m9,248[0m │
		├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
		│ max_pooling2d_2 ([38;5;33mMaxPooling2D[0m)       │ ([38;5;45mNone[0m, [38;5;34m15[0m, [38;5;34m15[0m, [38;5;34m32[0m)          │               [38;5;34m0[0m │
		├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
		│ dropout_3 ([38;5;33mDropout[0m)                  │ ([38;5;45mNone[0m, [38;5;34m15[0m, [38;5;34m15[0m, [38;5;34m32[0m)          │               [38;5;34m0[0m │
		├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
		│ conv2d_6 ([38;5;33mConv2D[0m)                    │ ([38;5;45mNone[0m, [38;5;34m15[0m, [38;5;34m15[0m, [38;5;34m64[0m)          │          [38;5;34m18,496[0m │
		├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
		│ conv2d_7 ([38;5;33mConv2D[0m)                    │ ([38;5;45mNone[0m, [38;5;34m13[0m, [38;5;34m13[0m, [38;5;34m64[0m)          │          [38;5;34m36,928[0m │
		├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
		│ max_pooling2d_3 ([38;5;33mMaxPooling2D[0m)       │ ([38;5;45mNone[0m, [38;5;34m6[0m, [38;5;34m6[0m, [38;5;34m64[0m)            │               [38;5;34m0[0m │
		├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
		│ dropout_4 ([38;5;33mDropout[0m)                  │ ([38;5;45mNone[0m, [38;5;34m6[0m, [38;5;34m6[0m, [38;5;34m64[0m)            │               [38;5;34m0[0m │
		├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
		│ flatten_1 ([38;5;33mFlatten[0m)                  │ ([38;5;45mNone[0m, [38;5;34m2304[0m)                │               [38;5;34m0[0m │
		├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
		│ dense_2 ([38;5;33mDense[0m)                      │ ([38;5;45mNone[0m, [38;5;34m512[0m)                 │       [38;5;34m1,180,160[0m │
		├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
		│ dropout_5 ([38;5;33mDropout[0m)                  │ ([38;5;45mNone[0m, [38;5;34m512[0m)                 │               [38;5;34m0[0m │
		├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
		│ dense_3 ([38;5;33mDense[0m)                      │ ([38;5;45mNone[0m, [38;5;34m10[0m)                  │           [38;5;34m5,130[0m │
		└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘
		[1m Total params: [0m[38;5;34m3,752,576[0m (14.31 MB)
		[1m Trainable params: [0m[38;5;34m1,250,858[0m (4.77 MB)
		[1m Non-trainable params: [0m[38;5;34m0[0m (0.00 B)
		[1m Optimizer params: [0m[38;5;34m2,501,718[0m (9.54 MB)

Code cell <5ym5P2XIeMYQ>
# %% [code]
loaded_model.evaluate(test_features, test_labels)
predictions = loaded_model.predict(test_features)
#predict_classes = loaded_model.predict_classes(test_features)

predictions
Execution output
1KB
	Stream
		[1m313/313[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m15s[0m 45ms/step - accuracy: 0.7613 - loss: 0.6799
		[1m313/313[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m19s[0m 62ms/step
	text/plain
		array([[5.6695886e-04, 5.1733651e-03, 5.9399419e-03, ..., 1.9263122e-03,
		        7.4487287e-03, 6.1846193e-04],
		       [3.5947275e-03, 3.8610199e-01, 3.1432359e-07, ..., 7.6983468e-09,
		        6.1005324e-01, 2.4854043e-04],
		       [2.0710558e-01, 1.5387499e-02, 5.3393519e-03, ..., 1.3818155e-03,
		        7.3431253e-01, 6.8748472e-03],
		       ...,
		       [1.8609488e-07, 5.1474082e-08, 7.5205165e-04, ..., 6.8672784e-03,
		        9.9046417e-07, 4.7796470e-07],
		       [3.8697187e-02, 8.5950333e-01, 2.4262439e-03, ..., 1.4157434e-03,
		        3.9290273e-04, 1.7558312e-03],
		       [4.9073540e-12, 2.1309521e-13, 2.6166305e-09, ..., 9.9949735e-01,
		        5.3883758e-13, 2.9143623e-12]], dtype=float32)

Code cell <xIIzerxPetUp>
# %% [code]
y_test = np.argmax(test_labels, axis=1)
y_pred = np.argmax(predictions, axis=1)



Code cell <3dZb1BRQiHZA>
# %% [code]
print(predictions[0])
clase = np.argmax(predictions[0])
clase

Text cell <gIlS_USfiLZN>
# %% [markdown]
##Metricas

Code cell <HZfgBS2xhbNE>
# %% [code]
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Asume que 'y_true' son las etiquetas verdaderas y 'y_pred' son las etiquetas predichas
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm) # Reemplaza con tus etiquetas de clase
disp.plot()
plt.show()
Execution output
65KB
	text/plain
		<Figure size 640x480 with 2 Axes>

Code cell <Mx_AosCnh6FQ>
# %% [code]
from sklearn.metrics import classification_report

# Asume que 'y_true' son las etiquetas verdaderas y 'y_pred' son las etiquetas predichas
report = classification_report(y_test, y_pred)

print(report)
Execution output
1KB
	Stream
		precision    recall  f1-score   support

		           0       0.76      0.81      0.79      1000
		           1       0.90      0.86      0.88      1000
		           2       0.78      0.55      0.64      1000
		           3       0.61      0.52      0.56      1000
		           4       0.73      0.74      0.74      1000
		           5       0.56      0.79      0.66      1000
		           6       0.76      0.86      0.81      1000
		           7       0.83      0.79      0.81      1000
		           8       0.89      0.85      0.87      1000
		           9       0.87      0.84      0.85      1000

		    accuracy                           0.76     10000
		   macro avg       0.77      0.76      0.76     10000
		weighted avg       0.77      0.76      0.76     10000

Text cell <KVtUzVwuiVB7>
# %% [markdown]
##Predicción

Code cell <-jowsEyreweh>
# %% [code]
import cv2
from google.colab.patches import cv2_imshow

def predecir(path, label):
  #Cargar imagen
  image = cv2.imread(path)

  #Redimensionar 32*32
  imagen_redimensionada = cv2.resize(image, (32, 32))

  #Mostrar imagen
  cv2_imshow(imagen_redimensionada)

  #Predecir la clase
  clase = np.argmax(loaded_model.predict(np.array([imagen_redimensionada])))
  predicted_label = class_names[clase]

  print(f"Etiqueta original: {label}")
  print(f"Etiqueta predicha: {predicted_label}")


Code cell <4joyHkY1eMiQ>
# %% [code]
rutas = [["/content/airplane_1.webp","airplane"],
         ["/content/airplane_2.jpg","airplane"],
         ["/content/bird_1.jpg","bird"],
         ["/content/bird_2.jpg","bird"],
         ["/content/cat_1.jpg","cat"],
         ["/content/cat_2.jpg","cat"]
]

for x in rutas:
  predecir(x[0], x[1])
Execution output
27KB
	Stream
		[1m1/1[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m0s[0m 27ms/step
		Etiqueta original: airplane
		Etiqueta predicha: airplane
		[1m1/1[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m0s[0m 25ms/step
		Etiqueta original: airplane
		Etiqueta predicha: airplane
		[1m1/1[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m0s[0m 23ms/step
		Etiqueta original: bird
		Etiqueta predicha: horse
		[1m1/1[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m0s[0m 25ms/step
		Etiqueta original: bird
		Etiqueta predicha: airplane
		[1m1/1[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m0s[0m 25ms/step
		Etiqueta original: cat
		Etiqueta predicha: horse
		[1m1/1[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m0s[0m 25ms/step
		Etiqueta original: cat
		Etiqueta predicha: horse
	text/plain
		<PIL.Image.Image image mode=RGB size=32x32>
		<PIL.Image.Image image mode=RGB size=32x32>
		<PIL.Image.Image image mode=RGB size=32x32>
		<PIL.Image.Image image mode=RGB size=32x32>
		<PIL.Image.Image image mode=RGB size=32x32>
		<PIL.Image.Image image mode=RGB size=32x32>

Text cell <YF55DA1Aku-z>
# %% [markdown]
Tarea

- Crear una red convolucional con el dataset MNIST
- Guardar el modelo con .save
- Hacer Prdicciones con imagenes
- Realizar metricas de evaluación

Code cell <by6tIBKak8hp>
# %% [code]


SyntaxError: invalid decimal literal (<ipython-input-1-1f8ef7cc939c>, line 1)